In [ ]:
#@title ön gereklilikler

!apt-get install pciutils nano neofetch
%cd /content
!git clone https://github.com/YKesX/Trident-RFooocus.git
%cd /content
!wget -O Trident-RFooocus/models/checkpoints/RealViz5.safetensors https://huggingface.co/SG161222/RealVisXL_V5.0/resolve/main/RealVisXL_V5.0_fp32.safetensors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Detay kontrol aktivasyonu için webui.py yaması (çalıştırmak yeterli)
WEBUI_PATH = "/content/Trident-RFooocus/webui.py"  # <-- change this to your path

import re
from pathlib import Path

p = Path(WEBUI_PATH)
assert p.exists(), f"File not found: {p}"

# 1) Restore from .bak if our previous patch created it (and file looks broken)
bak = p.with_suffix(".py.bak")
src = p.read_text(encoding="utf-8")

if "IndentationError" in src or "expected an indented block" in src:
    # (This won't usually be present in file text, but just in case.)
    pass

# If a backup exists, prefer restoring it first so we patch a clean file
if bak.exists():
    print(f"Restoring backup from: {bak.name}")
    src = bak.read_text(encoding="utf-8")

original = src
changes = 0

def replace_widget_visibility(name_regex):
    global src, changes
    # Turn "visible=False" to "visible=True" inside the widget construction
    # e.g., cfg = gr.Slider(..., visible=False, )
    pattern = rf'({name_regex}\s*=\s*gr\.[A-Za-z_]+\([^\)]*?)visible\s*=\s*False'
    repl    = r'\1visible=True'
    src, n = re.subn(pattern, repl, src, flags=re.DOTALL)
    changes += n

def force_visible_in_function(func_name):
    """Within a function body, flip any gr.update(visible=False) -> True."""
    global src, changes
    # Capture function body non-greedily up to the next 'def ' at column 0 OR a decorator OR the known sentinel lines.
    pattern = rf'(def\s+{func_name}\s*\([^\)]*\):\n)(.*?)(?=\n\s*def\s|\n\s*@|\Z)'
    m = re.search(pattern, src, flags=re.DOTALL)
    if not m:
        print(f"⚠️ Could not find function: {func_name}")
        return
    head, body = m.group(1), m.group(2)
    new_body, n1 = re.subn(r'gr\.update\(\s*visible\s*=\s*False\s*\)', 'gr.update(visible=True)', body)
    changes += n1
    # Also flip any tuple/dict uses: visible=False,
    new_body, n2 = re.subn(r'visible\s*=\s*False', 'visible=True', new_body)
    changes += n2
    src = src[:m.start(1)] + head + new_body + src[m.end(2):]

# 2) Make the core advanced widgets visible at definition
for w in [
    r'perf_name', r'perf_save', r'custom_steps',
    r'cfg', r'sampler_name', r'scheduler', r'clip_skip'
]:
    replace_widget_visibility(w)

# 3) Don’t let callbacks hide them
for fn in [
    'performance_changed',
    'preset_select',
    'preset_unselect',
    'preset_image_upload'
]:
    force_visible_in_function(fn)

# 4) Write patched file + keep a new backup of the restored original
safe_bak = p.with_suffix(".py.safe_orig")
if not safe_bak.exists():
    safe_bak.write_text(original, encoding="utf-8")
p.write_text(src, encoding="utf-8")

print(f"✅ Patched {p.name}.")
print(f"   • Original saved as: {safe_bak.name}")
print(f"   • Total targeted edits applied: {changes}")
print("➡️  Restart Trident-RFooocus and the advanced controls should be visible.")


In [ ]:
%cd /content/Trident-RFooocus
!python entry_with_update.py --share --gpu-only --nobrowser

In [ ]:
%cd /content
!cp -r 'Trident-RFooocus/outputs/2025-09-02/' drive/MyDrive/ColabExports/

In [ ]:
#@title Videodan kare çıkarıcı


BASE_DIR = "/content/drive/MyDrive/ColabExports/frames_clips"  # where your .mp4 lives
VIDEO_FILENAME = "clip_00-01-24_350_to_00-01-28_200.mp4"  # e.g. "myclip.mp4"; if "", auto-detects first video file in BASE_DIR
SAVE_AS_PNG = True   # if False, saves JPEGs
JPG_QUALITY = 95     # 50..100
PNG_COMPRESSION = 3  # 0..9 (lower = faster/larger)

import os, pathlib
from datetime import datetime
import cv2

# 0) Resolve video path (no upload, Drive assumed mounted already)
if not VIDEO_FILENAME:
    candidates = [f for f in os.listdir(BASE_DIR)
                  if f.lower().endswith((".mp4", ".mov", ".m4v", ".avi", ".mkv"))]
    if not candidates:
        raise RuntimeError(f"No video file found in {BASE_DIR}. Set VIDEO_FILENAME explicitly.")
    VIDEO_FILENAME = sorted(candidates)[0]

video_path = os.path.join(BASE_DIR, VIDEO_FILENAME)
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video not found: {video_path}")

# 1) Prepare output folder: BASE_DIR/clipframes/<video_stem> (create if needed)
OUTPUT_ROOT = os.path.join(BASE_DIR, "clipframes")
os.makedirs(OUTPUT_ROOT, exist_ok=True)

stem = pathlib.Path(video_path).stem
frames_dir = os.path.join(OUTPUT_ROOT, stem)
if os.path.exists(frames_dir) and os.listdir(frames_dir):
    frames_dir = f"{frames_dir}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(frames_dir, exist_ok=True)

print(f"📼 Video: {video_path}")
print(f"🗂️  Output folder: {frames_dir}")

# 2) Extract frames with OpenCV
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError(f"Failed to open video: {video_path}")

fps = cap.get(cv2.CAP_PROP_FPS)
total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"ℹ️  {w}x{h} @ {fps:.3f} FPS, {total} frames (reported)")

count = 0
while True:
    ok, frame = cap.read()
    if not ok:
        break
    out_name = f"{stem}_frame_{count:06d}.png" if SAVE_AS_PNG else f"{stem}_frame_{count:06d}.jpg"
    out_path = os.path.join(frames_dir, out_name)
    if SAVE_AS_PNG:
        cv2.imwrite(out_path, frame, [cv2.IMWRITE_PNG_COMPRESSION, int(PNG_COMPRESSION)])
    else:
        cv2.imwrite(out_path, frame, [cv2.IMWRITE_JPEG_QUALITY, int(JPG_QUALITY)])
    count += 1

cap.release()
print(f"✅ Done. Saved {count} frames to: {frames_dir}")
